In [ ]:
#!pip install simpletransformers
import pandas as pd
import os 
import torch
from simpletransformers.question_answering import QuestionAnsweringModel
import json



train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 
train= train.dropna() 

In [ ]:
def FormatTrain(train): 

    ## We need to properly format the data so it can be used in the model. 
    ## As explained by Sourav Kumar Goyal, the input format must be exactly 
    ''' 
    {
        'context': "This tweet sentiment extraction challenge is great",
        'qas': [
            {
                'id': "00001",
                'question': "positive",
                'answers': [
                    {
                        'text': "is great",
                        'answer_start': 43
                    }
                ]
            }
        ]
    }
    ]
    ''' 
    ## in order to be interpreted by the model 




    output = [] 
    for index, row in train.iterrows(): 


        ## Find the start and end points of the selected text within the text 
        answer_start = row.text.find(row.selected_text) 

        context = row.text
        question_id = row.textID 
        question = row.sentiment 

        answer = row.selected_text 

        ans = [] 
        qas = [] 

        ans.append({'text': answer, 'answer_start': answer_start}) 
        qas.append({'id': question_id, 'question': question, 'is_impossible': False, 'answers': ans})
        
        output.append({'context': context, 'qas': qas})

    return output


def FormatTest(test): 
    ## According to Sourav Kumar Goyal, the test data must be of the format 
    '''
    {
        'context': "Some context as a demo",
        'qas': [
            {'id': '0', 'question': 'neutral'}
        ]
    }
])''' 



    output = [] 
    for index, row in test.iterrows(): 
        qas = []
        
        context = row.text 
        question = row.sentiment 
        question_id = row.textID

        qas.append({'id': question_id, 'question': question}) 

        output.append({'context': context, 'qas': qas})
    return output



In [ ]:
formatted_train_data = FormatTrain(train) 
formatted_test_data = FormatTest(test) 

In [ ]:

model = QuestionAnsweringModel('distilbert', 'distilbert-base-uncased-distilled-squad', 
                               args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,
                                     'learning_rate': 3e-2,
                                     'num_train_epochs': 1,
                                     'max_seq_length': 128,
                                     'doc_stride': 64,
                                     'fp16': False,
                                    }, use_cuda = False
                              )
                              

In [ ]:
model.train_model(formatted_train_data) 

In [ ]:
prediction = model.predict(formatted_train_data) 
## Make a prediction, and convert it to a dataframe 
df = pd.DataFrame.from_dict(pred) 

submission['selected_text'] = df['answer'] 
submission['textID'] = train['textID'] 

submission.to_csv('BERT_submission.csv', index = False) 